# RAG From Scratch: Routing


In [ ]:
! pip install langchain_community tiktoken langchain-google-genai langchainhub chromadb langchain youtube-transcript-api pytube

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_ENDPOINT']='https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY']='LANGCHAIN_API_KEY'
os.environ['GOOGLE_API_KEY']='GOOGLE_API_KEY'

# Logical Routing-
Routing is based on what type of question is asked to LLM. Accoding to the question LLM decides which data source will relevant to fetch relevant information.

In [9]:
from typing import Literal # Litreal is the type of module which is used to restirct a variable to a specific set of values.
from langchain_core.prompts import ChatPromptTemplate  # ChatPrompTemplate - Is specifically designed for Chat based LLM
from langchain_core.pydantic_v1 import BaseModel,Field
from langchain_google_genai import ChatGoogleGenerativeAI

# Data Model-
# pydantic_model that validate the LLM's output. The datasource field can only be one of three values.
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )


# LLM with function call
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',temperature=0)
structured_llm=llm.with_structured_output(RouteQuery)

# Prompt
system="""
    You are a expert at routing user question to the appropriate data source.
    Based on programming language th question is referring to, route it to relevant data source.
"""

prompt=ChatPromptTemplate.from_messages(
      [
          ('system',system),
          ('human',"{question}")
      ]
)

# Define  Router
router = prompt | structured_llm

In [10]:
question="""
      Why does not following code work:
      from langchain_core.prompts import ChatPromptTemplate
      prompt=ChatPromptTemplate.from_messages(["human","speak in {language}"])
      prompt.invoke('French')
    """

result=router.invoke({'question':question})

In [12]:
result.datasource

'python_docs'

In [13]:
def choose_route(result):
  if 'python_docs' in result.datasource.lower():
    return 'chain for python_docs'
  elif 'js_docs' in result.datasource.lower():
    return 'chain for js_docs'
  else:
    return 'chain for golang_docs'

from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)

In [14]:
full_chain.invoke({"question":question})

'chain for python_docs'

# Semantic Routing
Accoding to user question LLM decides the which prompt is best suitable to fetch relevant information.

In [19]:
from langchain.utils.math import cosine_similarity
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda,RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# Physics teacher Prompt
physics_template="""
      You are very smart physics professor.\
      You are great at answering questions about physics in very concise and easy to understand manner.\
      When you don't know the answer to a question you admit that you don't know.
      Here is the question:{query}
      """
# Maths Teacher Prompt
math_template="""
      You are a very good mathematician. You are great at ansering math questions \
      You are very good because you are able to break down hard problem into their component parts,\
      answer the component parts, and put them together to answer the border question.
      Here is the question:{query}
"""

# Embed Prompt
Embeddings=GoogleGenerativeAIEmbeddings(model='models/embedding-001')
prompt_template=[physics_template,math_template]
prompt_embeddings=Embeddings.embed_documents(prompt_template)


# Route question to the Prompt
def prompt_router(input):
  # Embedded question
  query_embedding= Embeddings.embed_query(input['query'])
  # Compute Similarity
  similarity=cosine_similarity([query_embedding],prompt_embeddings)[0]
  most_similar=prompt_template[similarity.argmax()]
  # Chosen prompt
  print('Using math' if most_similar==math_template else 'Using Physics')
  return PromptTemplate.from_template(most_similar)


chain=(
    {'query':RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatGoogleGenerativeAI(model='gemini-1.5-flash')
    | StrOutputParser()
)

print(chain.invoke('What is black hole'))

Using Physics
A black hole is a region of spacetime with gravity so strong that nothing, not even light, can escape.  It's formed when a massive star collapses at the end of its life.


# Query Structuring for Metadata Filters

In [29]:
from langchain_community.document_loaders import YoutubeLoader

docs=YoutubeLoader.from_youtube_url('https://www.youtube.com/watch?v=pbAd8O1Lvm4').load()
docs[0].metadata

{'source': 'pbAd8O1Lvm4'}

In [30]:
import datetime
from typing import Literal, Optional, Tuple
from langchain_core.pydantic_v1 import BaseModel, Field

class TutorialSearch(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    content_search: str = Field(
        ...,
        description="Similarity search query applied to video transcripts.",
    )
    title_search: str = Field(
        ...,
        description=(
            "Alternate version of the content search query to apply to video titles. "
            "Should be succinct and only include key words that could be in a video "
            "title."
        ),
    )
    min_view_count: Optional[int] = Field(
        None,
        description="Minimum view count filter, inclusive. Only use if explicitly specified.",
    )
    max_view_count: Optional[int] = Field(
        None,
        description="Maximum view count filter, exclusive. Only use if explicitly specified.",
    )
    earliest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Earliest publish date filter, inclusive. Only use if explicitly specified.",
    )
    latest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Latest publish date filter, exclusive. Only use if explicitly specified.",
    )
    min_length_sec: Optional[int] = Field(
        None,
        description="Minimum video length in seconds, inclusive. Only use if explicitly specified.",
    )
    max_length_sec: Optional[int] = Field(
        None,
        description="Maximum video length in seconds, exclusive. Only use if explicitly specified.",
    )

    def pretty_print(self) -> None:
        for field in self.__fields__:
            if getattr(self, field) is not None and getattr(self, field) != getattr(
                self.__fields__[field], "default", None
            ):
                print(f"{field}: {getattr(self, field)}")

In [31]:
from langchain_core.prompts import ChatPromptTemplate

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
Given a question, return a database query optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
structured_llm = llm.with_structured_output(TutorialSearch)
query_analyzer = prompt | structured_llm

In [32]:
query_analyzer.invoke({"question": "rag from scratch"}).pretty_print()

content_search: rag from scratch
title_search: rag from scratch


In [33]:
query_analyzer.invoke(
    {"question": "videos on chat langchain published in 2023"}
).pretty_print()

content_search: chat langchain
title_search: chat langchain
earliest_publish_date: 2023-01-01
latest_publish_date: 2024-01-01


In [34]:
query_analyzer.invoke(
    {"question": "videos that are focused on the topic of chat langchain that are published before 2024"}
).pretty_print()

content_search: chat langchain
title_search: chat langchain
latest_publish_date: 1970-01-01


In [35]:
query_analyzer.invoke(
    {
        "question": "how to use multi-modal models in an agent, only videos under 5 minutes"
    }
).pretty_print()

content_search: how to use multi-modal models in an agent
title_search: multi-modal agent
max_length_sec: 300
